In [1]:
import json
import pandas as pd
import gzip
import math
from datetime import datetime

# To count frequency in lists
import collections

# For graphing
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

### Purpose: find review count for Amazon vs. non-Amazon products in the subcategories

In [2]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('/Users/yujinglai/Dropbox/Eugenie/data/new_amazon_cell_phone_accessories/meta_Cell_Phones_and_Accessories.json.gz')

# Drop irrelevant columns
df.drop(['image', 'feature', 'also_buy', 'also_view', 'similar_item', 'date', 'details', 'tech1', 'tech2', 'fit'], axis=1, inplace=True)

# Drop records with nan in category
df = df.dropna(subset=['category'])

In [3]:
# Load review data
# This file contains reviews for all categories in Cell Phone Electronics within the one-year time frame
reviews = pd.read_csv('/Users/yujinglai/Downloads/reviews_asin.csv')
reviews.drop(reviews.columns[0], axis=1, inplace=True) # Drop index column

def numImg(x): 
    if (type(x) is str):
        return x.count(',')
    return 0

def numVote(x):
    if (type(x) is str):
        return float(int(x.replace(',', '')))
    if (math.isnan(x)):
        return 0
    return x

# Process reviews
reviews['image'] = reviews['image'].apply(numImg) # Add image count
reviews['vote'] = reviews['vote'].apply(numVote) # Add vote count
reviews['reviewTime'] = reviews['reviewTime'].apply(lambda x: str(datetime.strptime(x, '%m %d, %Y').date()))

/Users/yujinglai/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
def ifAmazon (x):
    if x == 'Amazon' or x == 'AmazonBasics':
        return 1
    return 0

# Add amazon label to each product
df['amazon'] = df['brand'].apply(ifAmazon)

In [5]:
df.head()

,category,description,title,brand,rank,main_cat,asin,price,amazon
0,"[Cell Phones & Accessories, Accessories, Selfi...","[If you've searched monopod amazon, monopod ip...",90 Degree Twist-lock Extendable Pole for Photo...,Bonison,">#2,181,607 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,0220132968,NaN,0
1,"[Cell Phones & Accessories, Accessories, Scree...",NaN,Fairy Tales (Doodle Art),Fitquipment,">#7,432,623 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,0843165685,NaN,0
2,"[Cell Phones & Accessories, Accessories, Scree...","[, Item:Half-Black Galaxy S9 PlusScratch termi...","[2-Pack] Galaxy S9 Plus Screen Protector,Cafet...",Cafetec,">#2,969 in Cell Phones & Accessories (See Top ...",Cell Phones & Accessories,0857757652,$9.99,0
3,"[Cell Phones & Accessories, Accessories, Scree...","[, Item:Half-Black Galaxy S9 PlusScratch termi...","[2-Pack] Galaxy S9 Plus Screen Protector,Cafet...",Cafetec,">#2,969 in Cell Phones & Accessories (See Top ...",Cell Phones & Accessories,0865815194,$9.99,0
4,"[Cell Phones & Accessories, Accessories, Car A...",[Volume I focuses on the center of the body in...,Get Stronger by Stretching with Thera Band Vol...,Noa Spector-Flock,"920,439inMoviesTV(",Movies & TV,087127244X,NaN,0


In [5]:
# All records have the same category labels
cats = []
df['category'].apply(lambda x: cats.extend(x))
print("There are %d unique subcategories" % len(set(cats)))

# Get the top 10 subcategories as a list
cats = collections.Counter(cats)
cats = {k: v for k, v in reversed(sorted(cats.items(), key = lambda item: item[1]))}
cats = list(cats.keys())[0:1]
print("The top 10 subcategories are: %s" % cats)

There are 4911 unique subcategories
The top 10 subcategories are: ['Cell Phones & Accessories']


In [6]:
def common_member(a, cats): 
    a_set = set(a) 
    b_set = set(cats) 
    if (a_set & b_set): 
        return True 
    else: 
        return False

In [7]:
# Get products within top subcategories
df = df.loc[df['category'].apply(lambda x: common_member(x, cats))]

In [5]:
df.shape

(534651, 9)

In [19]:
def remove_uncommon(x, cats):
    return list(set(x) & set(cats))

In [20]:
# Remove subcategories not in tops
df['category'] = df['category'].apply(lambda x: remove_uncommon(x, cats))

In [21]:
df['category'].head()

0    [Cell Phones & Accessories]
1    [Cell Phones & Accessories]
2    [Cell Phones & Accessories]
3    [Cell Phones & Accessories]
4    [Cell Phones & Accessories]
Name: category, dtype: object

In [10]:
def create_cols(df):
    for i in cats:
        df[i] = df['category'].apply(lambda x: int(i in x))

In [11]:
# Create a binary column for each subcategory
create_cols(df)

In [13]:
# Count the number products in each subcategory
# Before dropping products with less than 10 reviews
df['asin'].groupby([df['amazon']]).count()

amazon
0    534606
1        45
Name: asin, dtype: int64

In [12]:
df_merged = pd.merge(reviews, df, on='asin')

In [29]:
df_merged.head(5)

,asin,verified,reviewText,overall,summary,unixReviewTime,reviewerName,reviewTime,vote,image,category,description,title,brand,rank,main_cat,price,amazon,Cell Phones & Accessories
0,7391002801,True,Highly recommend!!!,5.0,Five Stars,1475884800,Michael Marone,2016-10-08,0.0,0,[Cell Phones & Accessories],[Silver Elegant Butterfly Foot Ankle Chain Sum...,Silver Elegant Butterfly Foot Ankle Chain Summ...,Accessory,">#1,978,999 in Tools & Home Improvement (See t...",Tools & Home Improvement,NaN,0,1
1,7391002801,True,great,5.0,Five Stars,1464566400,tim jensen,2016-05-30,0.0,0,[Cell Phones & Accessories],[Silver Elegant Butterfly Foot Ankle Chain Sum...,Silver Elegant Butterfly Foot Ankle Chain Summ...,Accessory,">#1,978,999 in Tools & Home Improvement (See t...",Tools & Home Improvement,NaN,0,1
2,8288878881,True,Perfect from box to phone. Only original produ...,5.0,Five Stars,1478131200,Amazon Customer,2016-11-03,0.0,0,[Cell Phones & Accessories],[Safely charge your phone using the original a...,Samsung Galaxy S2 Phone OEM Official Travel US...,Samsung,">#245,551 in Cell Phones & Accessories (See To...",Cell Phones & Accessories,NaN,0,1
3,8288878881,True,"I regret buying this charger. Yes, it was chea...",3.0,Get What You Pay For,1475280000,Trusted Reviewer,2016-10-01,0.0,0,[Cell Phones & Accessories],[Safely charge your phone using the original a...,Samsung Galaxy S2 Phone OEM Official Travel US...,Samsung,">#245,551 in Cell Phones & Accessories (See To...",Cell Phones & Accessories,NaN,0,1
4,8288878881,True,Best charger I've ever had... Love it!,5.0,Five Stars,1474243200,MOM-MOM,2016-09-19,0.0,0,[Cell Phones & Accessories],[Safely charge your phone using the original a...,Samsung Galaxy S2 Phone OEM Official Travel US...,Samsung,">#245,551 in Cell Phones & Accessories (See To...",Cell Phones & Accessories,NaN,0,1


In [18]:
#df_merged['index'] = df_merged.index
df_merged['index'].groupby([df['amazon']]).count()

amazon
0.0    517440
1.0        43
Name: index, dtype: int64

In [13]:
# Remove asin with less than 10 reviews
df_merged['review_count'] = df_merged.groupby('asin')['asin'].transform('count') # Count reviews/asin
df_merged = df_merged.loc[df_merged['review_count'] > 10]

In [14]:
# Count the number reviews in each subcategory
reviews_count = df_merged.groupby([df_merged['amazon']]).sum().astype(int)[cats]

In [15]:
# Count the number products in each subcategory
cols = ['asin','amazon'] + cats
df_asin = df_merged[cols].drop_duplicates()
products_count = df_asin.groupby([df_asin['amazon']]).sum().astype(int)

In [16]:
print(reviews_count.iloc[0,0], reviews_count.iloc[1,0])

1824228 1632


In [17]:
print(products_count.iloc[0,0], products_count.iloc[1,0])

29276 9


In [18]:
cats[0]

'Cell Phones & Accessories'